In [ ]:
from astropy import units as u
from gammapy.datasets import Datasets
from gammapy.modeling.models import Models
from gammapy.modeling import Fit

import os
import sys
import importlib
path_my_modules = "/home/born-again/Documents/GitHub/my_modules"
module_path = os.path.abspath(f'{path_my_modules}/config')
if module_path not in sys.path:
    sys.path.append(module_path)

import cfg
importlib.reload(cfg)

module_path = os.path.abspath(f'{path_my_modules}/{cfg.dir_utilities}')
if module_path not in sys.path:
    sys.path.append(module_path)

import utilities as utl
importlib.reload(utl)

module_path = os.path.abspath(f'{path_my_modules}/{cfg.dir_plot_style}')
if module_path not in sys.path:
    print(sys.path.append(module_path))
    sys.path.append(module_path)

import plotter
importlib.reload(plotter)

module_path = os.path.abspath(f'{path_my_modules}/{cfg.dir_spectral_models}')
if module_path not in sys.path:
    sys.path.append(module_path)

import spectral_models as spec
importlib.reload(spec)

module_path = os.path.abspath(f'{path_my_modules}/{cfg.dir_lhaaso_analysis}')
if module_path not in sys.path:
    print(sys.path.append(module_path))
    sys.path.append(module_path)

import lhaaso_analysis as lhaaso
importlib.reload(lhaaso)

module_path = os.path.abspath(f'{path_my_modules}/{cfg.dir_gammapy_catalogs}')
if module_path not in sys.path:
    print(sys.path.append(module_path))
    sys.path.append(module_path)

import gammapy_catalogs as gammapy_cat
importlib.reload(gammapy_cat)

module_path = os.path.abspath(f'{path_my_modules}/{cfg.dir_cta_simulation}')
if module_path not in sys.path:
    print(sys.path.append(module_path))
    sys.path.append(module_path)

import cta_simulation as cta
importlib.reload(cta)

%matplotlib inline
import matplotlib.pyplot as plt # A collection of command style functions


path_my_plot_style = f"{path_my_modules}/{cfg.dir_plot_style}/my_plot_style.txt" 
plt.style.use(path_my_plot_style)

In [3]:
path_CTA_datasets = f"/home/born-again/Documents/GitHub/JCAP_2023-1/my_notebooks/pulsars/analysis/datasets/"

roi_name = "LHAASO_J1839-0545_roi_1.017deg_e_ref_min_100.0GeV"
datasets_name = "HESS_J1837-069-CTA-PSR_J1837-0604_50h"

In [4]:
path_file = f"{path_CTA_datasets}/{roi_name}/{datasets_name}"
datasets_joint =Datasets.read(filename=f"{path_file}/datasets{cfg.format_yaml}", filename_models=f"{path_file}/models{cfg.format_yaml}")

No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.


In [5]:
models_joint = Models()
models_joint.extend(datasets_joint.models)

In [ ]:
model =models_joint[-1]
models = [model]
datasets = Datasets(datasets_joint)

In [7]:
dict_leg_style = utl.load_dictionary(name=cfg.dict_leg_style, path_file=path_file)
dict_sep = utl.load_dictionary(name=cfg.dict_separation, path_file=path_file)

In [10]:
utl.load_dictionary(name=cfg.dict_separation, path_file=path_file)

{}

In [11]:
dict_leg_style

{'HESS J1837-069: gamma-cat': ('aqua', 's'),
 'HESS J1837-069: hgps': ('fuchsia', 'o'),
 '2HWC J1837-065': ('plum', 's'),
 'LHAASO J1839-0545': ('red', 'o'),
 'CTA - PSR J1837-0604 (50h)': ('blue', 's')}

In [ ]:
dict_leg_style = plotter.set_leg_style(dict_leg_style, models = models)
dict_plot_limits = dict(
    energy_bounds = [5e-2, 2e3] * u.TeV,
    ylim = [1e-15, 1e-9]
)
plotter.plot_SED(
    name = "model_name", 
    dict_plot_limits=dict_plot_limits,
    datasets=datasets, 
    models=models, 
    dict_leg_style=dict_leg_style, 
)

In [ ]:
dataset_CTA = datasets_joint[-1].copy()
sky_model_CTA_in = model.copy(name=f"{datasets_joint[-1].name} {model.spectral_model.tag[1]}")


In [ ]:
model = spec.sky_model_ecpl()
datasets.models = [model]
fit_joint = Fit()
result_joint = fit_joint.run(datasets=datasets)
print(result_joint)

In [ ]:
from astropy import units as u
import matplotlib.pyplot as plt
import naima
from gammapy.modeling.models import Models, NaimaSpectralModel, SkyModel

particle_distribution = naima.models.ExponentialCutoffPowerLaw(
    1e30 / u.eV, 10 * u.TeV, 3.0, 30 * u.TeV
)
# radiative_model = naima.radiative.InverseCompton(
#     particle_distribution,
#     seed_photon_fields=["CMB", ["FIR", 26.5 * u.K, 0.415 * u.eV / u.cm**3]],
#     Eemin=100 * u.GeV,
# )

radiative_model = naima.models.PionDecay(particle_distribution)

model = NaimaSpectralModel(radiative_model, distance=1.5 * u.kpc)

In [ ]:
print(model.spectral_model)

In [ ]:
sky_model_naima = SkyModel(
    spectral_model=model,
    name="naima")

In [ ]:
model = sky_model_naima
datasets.models = [model]
fit_joint = Fit(backend="sherpa")
result_joint = fit_joint.run(datasets=datasets)
print(result_joint)

In [ ]:
print(model)

In [ ]:
display(model.parameters.to_table())

In [ ]:
dict_leg_style = plotter.set_leg_style(dict_leg_style, models = [model])
dict_plot_limits = dict(
    energy_bounds = [5e-2, 2e3] * u.TeV,
    ylim = [1e-25, 1e-9]
)
plotter.plot_SED(
    name = "model_name", 
    dict_plot_limits=dict_plot_limits,
    datasets=datasets, 
    models=[model, sky_model_CTA_in], 
    dict_leg_style=dict_leg_style, 
)

In [ ]:
print(datasets_joint)

In [ ]:
list_data =[]
names = ["e_ref", "dnde", "dnde_errp", "dnde_errn", "is_ul"]
new_names = ["energy", "flux", "flux_error_hi", "flux_error_lo", "ul"]

In [ ]:
for index, dataset in enumerate(datasets_joint):
    table = dataset.data.to_table(sed_type="dnde")
    table.rename_columns(names, new_names)
    list_data.append(table[new_names])

In [ ]:
list_data

In [ ]:
#!/usr/bin/env python
import astropy.units as u
import numpy as np
from astropy.io import ascii

import naima
from naima.models import ExponentialCutoffPowerLaw, InverseCompton

## Read data

# data = ascii.read("RXJ1713_HESS_2007.dat")


def ElectronIC(pars, data):
    """
    Define particle distribution model, radiative model, and return model flux
    at data energy values
    """

    ECPL = ExponentialCutoffPowerLaw(
        pars[0] / u.eV, 10.0 * u.TeV, pars[1], 10 ** pars[2] * u.TeV
    )
    IC = InverseCompton(ECPL, seed_photon_fields=["CMB"])

    return IC.flux(data, distance=1.0 * u.kpc)


def lnprior(pars):
    # Limit amplitude to positive domain
    logprob = naima.uniform_prior(pars[0], 0.0, np.inf)
    return logprob

In [ ]:

## Set initial parameters and labels
p0 = np.array((1e30, 3.0, np.log10(30)))
labels = ["norm", "index", "log10(cutoff)"]

## Run sampler
sampler, pos = naima.run_sampler(
    data_table=list_data,
    p0=p0,
    labels=labels,
    model=ElectronIC,
    prior=lnprior,
    nwalkers=32,
    nburn=100,
    nrun=20,
    threads=4,
    prefit=True,
    interactive=False,
)


In [ ]:
## Save run results
out_root = "RXJ1713_IC_minimal"
naima.save_run(out_root + ".hdf5", sampler)

## Save diagnostic plots and results table
naima.save_diagnostic_plots(out_root, sampler, sed=False)
naima.save_results_table(out_root, sampler)